In [59]:
import pandas as pd 
  
# intialise data of lists. 
data = {'A':[1.2,1.4,2.5,3.3], 
        'B':[2.3,1.1,3.0,2.7],
        'C':[2.5,2.5,2.5,2.5],
        'D':['L','R','R','L']} 
  
# Create DataFrame 
df = pd.DataFrame(data) 
dft = df.copy()  
# Print the output. 
df 

,A,B,C,D
0,1.2,2.3,2.5,L
1,1.4,1.1,2.5,R
2,2.5,3.0,2.5,R
3,3.3,2.7,2.5,L


In [60]:
from sklearn import preprocessing

# creating instance of labelencoder
labelencoder = preprocessing.LabelEncoder()
labelencoder.fit(df['D'])
print(list(labelencoder.classes_))

# Assigning numerical values and storing in another column
df['D'] = labelencoder.fit_transform(df['D'])
df

['L', 'R']


,A,B,C,D
0,1.2,2.3,2.5,0
1,1.4,1.1,2.5,1
2,2.5,3.0,2.5,1
3,3.3,2.7,2.5,0


In [61]:
### skfuzzy.membership.gaussmf 

def gaussmf(x, mean, sigma):
    """
    Gaussian fuzzy membership function.

    Parameters
    ----------
    x : 1d array or iterable
        Independent variable.
    mean : float
        Gaussian parameter for center (mean) value.
    sigma : float
        Gaussian parameter for standard deviation.

    Returns
    -------
    y : 1d array
        Gaussian membership function for x

    """
    eps=1e-16
    
    return np.exp(-((x - mean) ** 2.) / (float(sigma) ** 2.+eps))

In [62]:
datasets = {}
by_class = df.groupby('D')

for groups, data in by_class:
    datasets[groups] = data

In [209]:
datasets

{0:      A    B    C  D
 0  1.2  2.3  2.5  0
 3  3.3  2.7  2.5  0,
 1:      A    B    C  D
 1  1.4  1.1  2.5  1
 2  2.5  3.0  2.5  1}

In [63]:
len(datasets)
meand=[]
stdd=[]
for i in range(len(datasets)):
    print(datasets[i])
    meand.append(list(datasets[i].mean(axis = 0)))
    print('mean',meand)
    stdd.append(list(datasets[i].std(axis = 0)))
    print('std',stdd)

     A    B    C  D
0  1.2  2.3  2.5  0
3  3.3  2.7  2.5  0
mean [[2.25, 2.5, 2.5, 0.0]]
std [[1.4849242404917498, 0.2828427124746193, 0.0, 0.0]]
     A    B    C  D
1  1.4  1.1  2.5  1
2  2.5  3.0  2.5  1
mean [[2.25, 2.5, 2.5, 0.0], [1.95, 2.05, 2.5, 1.0]]
std [[1.4849242404917498, 0.2828427124746193, 0.0, 0.0], [0.7778174593052023, 1.34350288425444, 0.0, 0.0]]


In [214]:
eps=1e-16
f=np.exp(-((2.3 - 2.5) ** 2.) / (float(0.2929) ** 2.+eps))
f

0.6273488082851381

In [64]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np

X=df.to_numpy()
r,c=X.shape

oldf=df.columns
newf=list(range(c))
df.rename(columns=dict(zip(oldf, newf)), inplace=True) #renaming column
D=list(df[newf[-1]])

labelencoder.fit(df[newf[-1]])
list(labelencoder.classes_)
classes=len(list(labelencoder.classes_))
FD=np.zeros((r,(c-1)*classes))

for i in range(len(datasets)):
    oldf=datasets[i].columns
    newf=list(range(c))
    datasets[i].rename(columns=dict(zip(oldf, newf)), inplace=True) #renaming column
    print(datasets[i])

for j in range(c-1):
    for i in range(r):
        l=((j+1)*classes)-1
        for k in range(classes):
            g=gaussmf(X[i][j], np.array(meand[k][j]), np.array(stdd[k][j]))
            #print(g)
            FD[i][l-1]=g #float(str(round(g, 8)))
            l = l+1
            
FD

     0    1    2  3
0  1.2  2.3  2.5  0
3  3.3  2.7  2.5  0
     0    1    2  3
1  1.4  1.1  2.5  1
2  2.5  3.0  2.5  1


array([[6.06530660e-01, 3.94651546e-01, 6.06530660e-01, 9.65966583e-01,
        1.00000000e+00, 1.00000000e+00],
       [7.20604815e-01, 6.06530660e-01, 2.28973485e-11, 6.06530660e-01,
        1.00000000e+00, 1.00000000e+00],
       [9.72053270e-01, 6.06530660e-01, 4.39369336e-02, 6.06530660e-01,
        1.00000000e+00, 1.00000000e+00],
       [6.06530660e-01, 4.91736831e-02, 6.06530660e-01, 7.91304822e-01,
        1.00000000e+00, 1.00000000e+00]])

In [65]:
D=list(df[newf[-1]])
D

[0, 1, 1, 0]

In [66]:
D=np.array(D)
D.shape[0]

4

In [67]:
D=D.reshape(4,1)

In [68]:
F=np.concatenate((FD, D), axis=1)
F

array([[6.06530660e-01, 3.94651546e-01, 6.06530660e-01, 9.65966583e-01,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [7.20604815e-01, 6.06530660e-01, 2.28973485e-11, 6.06530660e-01,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [9.72053270e-01, 6.06530660e-01, 4.39369336e-02, 6.06530660e-01,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [6.06530660e-01, 4.91736831e-02, 6.06530660e-01, 7.91304822e-01,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [69]:
# Create Fuzzyfied DataFrame 
fdf = pd.DataFrame(F) 
  
# Print the output. 
fdf 

,0,1,2,3,4,5,6
0,0.606531,0.394652,6.065307e-01,0.965967,1.0,1.0,0.0
1,0.720605,0.606531,2.289735e-11,0.606531,1.0,1.0,1.0
2,0.972053,0.606531,4.393693e-02,0.606531,1.0,1.0,1.0
3,0.606531,0.049174,6.065307e-01,0.791305,1.0,1.0,0.0


In [70]:
def fuzzy_distance10(x,y):
    eps = 1e-16
    d = 1-(sum(min(x,y))/(sum(min(x,y))+eps))
    
    return d

def getthefuzzyvalues(F,p,t,cls):
    
    temp=[]
    for i in range(cls):
        temp.append(F[p][((t)*cls)+i])
        
    return temp

In [71]:
S=[]
for i in range(r):
    for j in range(r):
        if i>j and F[i][-1]!=F[j][-1]:
            dv=[]
            for k in range(c-1):
                x=getthefuzzyvalues(F,i,k,classes)
                y=getthefuzzyvalues(F,j,k,classes)
                dist=fuzzy_distance10(x,y)
                #dist=float(str(round(dist, 4)))
             
                dv.append(dist)
        
            S.append(dv)

print('\nValid entries in Discernibilty Matrix: ',S)   


Valid entries in Discernibilty Matrix:  [[0.0, 2.220446049250313e-16, 0.0], [0.0, 2.220446049250313e-16, 0.0], [2.220446049250313e-16, 2.220446049250313e-16, 0.0], [2.220446049250313e-16, 2.220446049250313e-16, 0.0]]


In [72]:
A=np.mean(S,axis=0)
A

array([1.11022302e-16, 2.22044605e-16, 0.00000000e+00])

In [51]:
np.argsort(A)

array([2, 0, 1], dtype=int64)

In [52]:
ids = A.argsort()[::-1][:2]
ids

array([1, 0], dtype=int64)

In [73]:
best=2
ids=np.flip(np.argsort(A))
reduct=ids[:best]
reduct

array([1, 0], dtype=int64)

In [74]:
idx=list(reduct)
idx.append(newf[-1])
df[idx]

,1,0,3
0,2.3,1.2,0
1,1.1,1.4,1
2,3.0,2.5,1
3,2.7,3.3,0


In [25]:
#### Calling My Fuuzy Discernibility Matrix-based k-best discriminating features selection Algorithm ###

import pandas as pd 
import time
import my_fuzzy_discrn_mat

######### Entry Main ##########    
# intialise data of lists. 
data = {'A':[1.2,1.4,2.5,3.3], 
        'B':[2.3,1.1,2.0,2.7],
        'C':[2.5,2.5,2.5,2.5],
        'D':['L','R','R','L']} 
  
# Create DataFrame 
df = pd.DataFrame(data) 
best=2

# Print the output. 
print('\nBefore Reduct Computation: \n',df) 
t1=time.time()
rdf, rf = my_fuzzy_discrn_mat.my_fuzzy_discrn_mat(df,best) 
t2=time.time()-t1
print('\nAfter Reduct Computation: \n',rdf) 
print('\nObtained Reduct: {}-best discriminating features {}'.format(best,rf))
print('\nTotal execution time for FDM: {:.6f} seconds'.format(t2))


Before Reduct Computation: 
      A    B    C  D
0  1.2  2.3  2.5  L
1  1.4  1.1  2.5  R
2  2.5  2.0  2.5  R
3  3.3  2.7  2.5  L

After changing the column names to 0 to ...
      0    1    2  3
0  1.2  2.3  2.5  L
1  1.4  1.1  2.5  R
2  2.5  2.0  2.5  R
3  3.3  2.7  2.5  L

After Reduct Computation: 
      1    0  3
0  2.3  1.2  L
1  1.1  1.4  R
2  2.0  2.5  R
3  2.7  3.3  L

Obtained Reduct: 2-best discriminating features [1 0]

Total execution time for FDM: 0.005956 seconds
